<a href="https://colab.research.google.com/github/judem-21/Natural-Language-Processing/blob/main/Sentiment%20Analysis/SAModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==

In [ ]:
import pandas as pd
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.optim.lr_scheduler import StepLR

In [ ]:
dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

train_df.csv:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

val_df.csv:   0%|          | 0.00/601k [00:00<?, ?B/s]

test_df.csv:   0%|          | 0.00/586k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31232 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5205 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5206 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def filter_none(example):
    return example["text"] is not None

dataset['test'] = dataset['test'].filter(filter_none)
dataset['validation']=dataset['validation'].filter(filter_none)
dataset['train']=dataset['train'].filter(filter_none)

Filter:   0%|          | 0/5206 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5205 [00:00<?, ? examples/s]

Filter:   0%|          | 0/31232 [00:00<?, ? examples/s]

In [ ]:
def tokenize(batch):
    tokenised = tokenizer(text=batch["text"], truncation=True)
    tokenised["labels"] = batch["label"]
    return tokenised

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['id', 'text', 'sentiment'])

Map:   0%|          | 0/31232 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=32,
    shuffle=True,
    collate_fn=data_collator
)

In [ ]:
for batch in train_loader:
    print({k: v.shape for k, v in batch.items()})  # Shows input_ids, attention_mask shapes
    print(f'Input Ids:{batch["input_ids"]}')
    print(f'Attention Mask:{batch["attention_mask"]}')
    print(f'Labels:{batch["labels"]}')
    break

{'input_ids': torch.Size([32, 91]), 'token_type_ids': torch.Size([32, 91]), 'attention_mask': torch.Size([32, 91]), 'labels': torch.Size([32])}
Input Ids:tensor([[  101, 10166,  1012,  ...,     0,     0,     0],
        [  101,  2017,  2064,  ...,     0,     0,     0],
        [  101,  2026, 10424,  ...,     0,     0,     0],
        ...,
        [  101,  2204,  1011,  ...,     0,     0,     0],
        [  101,  2293,  2009,  ...,     0,     0,     0],
        [  101,  1045,  2001,  ...,  2490,  1012,   102]])
Attention Mask:tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])
Labels:tensor([1, 1, 2, 1, 2, 2, 1, 2, 2, 0, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 0, 2, 1, 1,
        2, 0, 1, 2, 1, 0, 2, 0])


In [ ]:
class SentimentAnalyser(nn.Module):
  def __init__(self,embedding_dim,vocab_size,num_rec_layers,dropout,pad_idx,bidirectional=True):
    super().__init__()
    #self.embedding_dim=embedding_dim
    self.embeddings=nn.Embedding(vocab_size,embedding_dim,padding_idx=pad_idx)
    self.linear=nn.Linear(2*embedding_dim if bidirectional else embedding_dim,3)
    self.lstm=nn.LSTM(input_size=embedding_dim,num_layers=num_rec_layers,hidden_size=embedding_dim,bidirectional=bidirectional,batch_first=True,dropout=dropout)
    self.dropout=nn.Dropout(dropout)

  def forward(self,input):
    input_embeddings=self.embeddings(input)
    out,(_,_)=self.lstm(input_embeddings)
    out=self.linear(self.dropout(out[:,-1,:]))
    return out

In [ ]:
model=SentimentAnalyser(embedding_dim=512,vocab_size=tokenizer.vocab_size,num_rec_layers=4,dropout=0.5,pad_idx=pad_idx).to(device)

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs,lr=1000,0.1
pad_idx = tokenizer.pad_token_id
# Learning rate scheduler: reduce LR by gamma every 'step_size' epochs
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)  # halve LR every 5 epochs
loss_fn=nn.CrossEntropyLoss()

In [ ]:
epoch_losses=[]
for epoch in range(1,num_epochs+1):
  batch_losses=[]
  for idx,batch in enumerate(train_loader):
    optimizer.zero_grad()
    logits=model(batch['input_ids'].to(device))
    loss=loss_fn(logits,batch['labels'].to(device))
    loss.backward()
    optimizer.step()
    batch_losses.append(loss.item())
    if (epoch==1 or epoch%100==0) and (idx+1)%100==0:print(f'Epoch {epoch}, Batch {idx+1}, Batch Loss:{batch_losses[-1]:.8f}')
  epoch_losses.append(np.mean(batch_losses))
  scheduler.step()
  print(f'Epoch:{epoch}, Learning Rate={scheduler.get_last_lr()[0]}')
  if epoch==1 or epoch%100==0:print(f'Epoch:{epoch}, Epoch Loss:{epoch_losses[-1]:.4f}')